/*
 * Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
 * SPDX-License-Identifier: MIT-0
 *
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this
 * software and associated documentation files (the "Software"), to deal in the Software
 * without restriction, including without limitation the rights to use, copy, modify,
 * merge, publish, distribute, sublicense, and/or sell copies of the Software, and to
 * permit persons to whom the Software is furnished to do so.
 *
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,
 * INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A
 * PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
 * HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION
 * OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
 * SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
 */

### Replication of data from a Kinesis Data Stream in `us-east-1` to another Kinesis Data Stream in `us-west-1`
If you are attempting to apply this to your workloads, please ensure you make the following modifications:
- Modify the schema specified in both tables to match your incoming data
- Change the `stream` name in both tables to match both your source stream and destination stream
- Ensure that your Kinesis Data Analytics application is running in the *same* region as your **source** stream to minimize latency and disruptions when replicating.


Please note that this replication process does not factor in any other **necessary** replication considerations such as failover detection, client failover and offset synchronization between clients. 

In [1]:
%flink.ssql(type=update)

DROP TABLE IF EXISTS my_source_stream;
CREATE TABLE my_source_stream (
    data STRING
)
WITH (
  'connector' = 'kinesis',
  'stream' = 'us-east-1-stream',
  'aws.region' = 'us-east-1',
  'scan.stream.initpos' = 'LATEST',
  'format' = 'raw'
); 

In [2]:
%flink.ssql(type=update)
DROP TABLE IF EXISTS my_sink_stream;

CREATE TABLE my_sink_stream (
    data STRING
)
WITH (
  'connector' = 'kinesis',
  'stream' = 'us-west-1-stream',
  'aws.region' = 'us-west-1',
  'format' = 'raw',
  'scan.stream.initpos' = 'TRIM_HORIZON'
); 

### Run the insert statement which will trigger the Apache Flink job to execute. 
This paragraph will take data from `my_source_stream` (us-east-1) and write it into the secondary stream `my_sink_stream` (us-west-1)


In [4]:
%flink.ssql(type=update)
INSERT INTO my_sink_stream select * from my_source_stream;

#### We can validate that all of our records have made it to the sink stream by reading from the beginning of the sink and counting all the records that have arrived thus far.
Delete the below paragraph ahead of deploying this notebook as a fully-fledged Apache Flink application on Kinesis Data Analytics for Apache Flink.

In [6]:
%flink.ssql(type=update)
SELECT COUNT(*) AS record_count FROM my_sink_stream

 
### Please ensure you stop all paragraphs OR stop the Flink application in the AWS Management Console once replication is no longer needed to free up resources.